In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_context('talk')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

cwd = os.getcwd().split('\\')
local_repo_path = '\\'.join(cwd[:-1])
raw_data_path = local_repo_path + r'\\data\\raw'
processed_data_path = local_repo_path + r'\\data\\processed'
feature_path = local_repo_path + r'\\data\\features'
model_comparison_path = local_repo_path + r'\\model_submissions'

In [3]:
output_feature_fname = '2degree_poly'
sj_feature_train = pd.read_csv(feature_path + r'\\sj_train_' + output_feature_fname + '.csv', index_col = 0)
sj_feature_test = pd.read_csv(feature_path + r'\\sj_test_' + output_feature_fname + '.csv', index_col = 0)
iq_feature_train = pd.read_csv(feature_path + r'\\iq_train_' + output_feature_fname + '.csv', index_col = 0)
iq_feature_test = pd.read_csv(feature_path + r'\\iq_test_' + output_feature_fname + '.csv', index_col = 0)

In [4]:
sj_feature_train.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,total_cases
0,0.0,-1.081814,-1.183554,-0.759571,-1.272727,-0.542448,-1.008038,-1.077631,-0.947159,-1.092417,...,-0.808394,-1.225806,-0.540115,-0.740437,-0.935945,-0.510820,-1.194577,-0.517934,-0.305917,4
1,0.0,-0.559956,-0.583693,-0.214395,-0.272727,-0.541333,-0.544774,-0.559237,-0.448256,-0.465126,...,-0.263413,-0.290432,-0.537789,-0.214041,-0.224612,-0.499007,-0.269139,-0.495076,-0.305623,5
2,0.0,0.006304,-0.005017,-0.214395,0.000000,-0.530953,0.006414,0.005247,-0.085027,-0.024420,...,-0.211121,-0.000225,-0.526301,-0.214041,-0.089200,-0.489124,0.000000,-0.479202,-0.302821,4
3,0.0,-0.061121,-0.053519,0.110260,0.227273,-0.510605,-0.061864,-0.061176,-0.011965,0.020094,...,0.079649,0.199738,-0.506222,0.111624,0.172680,-0.462490,0.229765,-0.454182,-0.296981,3
4,0.0,0.144991,0.146063,0.741194,0.500000,-0.479789,0.149107,0.144828,0.357315,0.254905,...,0.672788,0.461266,-0.475140,0.770548,0.609026,-0.416591,0.512061,-0.416642,-0.287263,6


In [11]:
# For supervised learning

from math import sqrt
from numpy import concatenate
import matplotlib.pyplot as plt

def series_to_supervise(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [12]:
n_lookback = 52

In [13]:
X_tr_sj_wide = series_to_supervise(sj_feature_train.drop('total_cases', axis=1), n_lookback, 1)
X_tr_sj_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis=1, inplace=True)
X_tr_sj_wide['total_cases'] = sj_feature_train['total_cases']
X_tr_sj_wide.to_csv(feature_path + r'\\sj_train_Lookback' + str(n_lookback) + '.csv')
X_tr_sj.wide.shape

In [ ]:
X_test_sj_wide = sj_feature_train.drop(['total_cases'], axis = 1).iloc[-n_lookback:,:].append(sj_feature_test.drop(['city','weekofyear', 'year'], axis = 1)).reset_index(drop = True)
X_test_sj_wide = series_to_supervised(X_test_sj_wide, n_lookback, 1)
X_test_sj_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
features_out = pd.DataFrame(X_test_sj_wide).reset_index(drop = True)
features_out['city'] = 'sj'
features_out['year'] = sj_feature_test['year']
features_out['weekofyear'] = sj_feature_test['weekofyear']
features_out.to_csv(feature_path + '\\sj_test_Lookback' + str(n_lookback) + '.csv')
X_test_sj_wide = X_test_sj_wide.values.reshape((X_test_sj_wide.shape[0], 1, X_test_sj_wide.shape[1]))
X_test_sj_wide.shape

In [ ]:
X_tr_iq_wide = series_to_supervised(iq_feature_train.drop('total_cases', axis = 1), n_lookback, 1)
X_tr_iq_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
X_tr_iq_wide['total_cases'] = iq_feature_train['total_cases']
X_tr_iq_wide.to_csv(feature_path + '\\iq_train_Lookback' + str(n_lookback) + '.csv')
X_tr_iq_wide.shape

In [ ]:
X_test_iq_wide = iq_feature_train.drop('total_cases', axis = 1).iloc[-n_lookback:,:].append(iq_feature_test.drop(['city','weekofyear', 'year'], axis = 1)).reset_index(drop = True)
X_test_iq_wide = series_to_supervised(X_test_iq_wide, n_lookback, 1)
X_test_iq_wide.drop(['var1(t)', 'var2(t)', 'var3(t)', 'var4(t)', 'var5(t)'], axis = 1, inplace = True)
features_out = pd.DataFrame(X_test_iq_wide).reset_index(drop = True)
features_out['city'] = 'iq'
features_out['year'] = iq_feature_test['year']
features_out['weekofyear'] = iq_feature_test['weekofyear']
features_out.to_csv(feature_path + '\\iq_test_Lookback' + str(n_lookback) + '.csv')
X_test_iq_wide = X_test_iq_wide.values.reshape((X_test_iq_wide.shape[0], 1, X_test_iq_wide.shape[1]))
X_test_iq_wide.shape